In [ ]:
import random
import logging
from collections import namedtuple,Counter

POPULATION_SIZE = 100
OFFSPRING_SIZE = 50
NUM_GENERATIONS = 1000

logging.basicConfig(format="%(message)s", level=logging.INFO)
Individual = namedtuple("Individual", ["selected","solution","available"])

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def fitness(state):
    #selected=state[0]
    solution = state[0]
    #return len(set(newlist) & selected), -len(set(newlist) | selected)
    cnt = Counter()
    cnt.update(sum((e for e in solution), start=()))
    return  sum(cnt[c] == 1 for c in cnt),0

def tournament(population,tournament_size=2):
    return max(random.choices(population,k=tournament_size), key=lambda i:fitness(i))
def mutation(state):
    solution,available=state  #selected,
    removeTuple=random.choice(solution) if len(solution)>0 else ()
    #if()
    addTuple=random.choice(available) if len(available)>0 else ()
    #selected=selected-set(removeTuple)
    #selected=selected|set(addTuple)
    available=tuple(x for x in available if x!=addTuple)
    solution=(*(x for x in solution if x!=removeTuple),addTuple)
    return (solution,available)  #selected,

def crossover(p1,p2):
    solution1, available = p1
    solution2, _ = p2
    cut1=random.randint(0,len(solution1))
    cut2 = random.randint(0, len(solution2))

    solution = tuple(set((*solution1[: cut1], *solution2[cut2 :])))
    newAvailable=tuple((set(solution1)|set(available))-set(solution))
    return (solution, newAvailable)


for N in [5, 10, 20, 100,500,1000]:
    lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    tuples = tuple(tuple(_) for _ in set(tuple(l) for l in lists))

    #tuples = tuple(tuple(sublist) for sublist in filteredLists)
    #print('\ntuples: ',tuples,'\n')

    population = list()
    # generate initial population, random
    for genome in [tuple(random.choices(tuples,k=random.randint(1,len(tuples)))) for _ in range(POPULATION_SIZE)]:
        #selected=set()
        #for _ in genome:
        #    selected=selected | set(_)
        available=tuple(set(tuples)-set(genome))
        population.append((tuple(set(genome)),available)) #eve add fitness  selected,

    fitness_log = [(0, fitness(i)) for i in population]

    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < 0.2:
                p = tournament(population)
                o = mutation(p)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = crossover(p1, p2)
            f = fitness(o)
            fitness_log.append((g + 1, f))
            offspring.append(o)
        population += offspring
        # sort and select the fittest mu
        population = sorted(population, key=lambda i: fitness(i), reverse=True)[:POPULATION_SIZE]
        if(fitness(population[0])[0]==N):
            break
    solution=population[0][0]
    print("solution found: ",solution)
    print(f"Solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution) - N) / N * 100:.0f}%)")

solution found:  ((0, 2), (1, 3), (4,))
Solution for N=5: w=5 (bloat=0%)
solution found:  ((0, 4, 7), (1, 6), (8, 2, 3), (9, 5))
Solution for N=10: w=10 (bloat=0%)
solution found:  ((0, 3, 5, 8, 9, 10, 13, 14, 17), (4, 7, 11, 12, 15, 16, 18), (16, 9, 19, 6), (1, 2, 9, 7))
Solution for N=20: w=24 (bloat=20%)
solution found:  ((97, 2, 34, 70, 71, 38, 72, 10, 75, 12, 16, 81, 83, 52, 86, 57, 29, 31), (1, 10, 11, 14, 18, 20, 21, 24, 31, 32, 33, 38, 39, 48, 49, 50, 52, 54, 58, 64, 66, 68, 74, 75, 77, 82, 84, 87, 88, 90, 91, 93, 96, 98), (0, 1, 4, 7, 10, 12, 13, 15, 19, 23, 25, 32, 38, 42, 44, 45, 51, 52, 55, 58, 59, 62, 65, 67, 78, 79, 84, 92, 98), (3, 5, 6, 17, 26, 29, 31, 35, 36, 40, 42, 43, 53, 56, 66, 69, 76, 82, 85, 92, 93, 94, 96, 97))
Solution for N=100: w=105 (bloat=5%)
